In [ ]:
import uuid
from IPython.display import display, HTML

import json

class RenderJSON:
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from renderer import render, SubfigureRenderer
import os
from benchart import BenchArt
from loader import Loader
from discards import *

def data_def(all_data):
    return all_data['data']['pgbench']['progress']

def metadata_def(all_data):
    return all_data['metadata']

loader = Loader('run_data')

loader.discard(discard_disk_vm, disk='p30', vm='Standard_D16ds_v4', invert=True)

runs = loader.run(data_def, metadata_def)

benchart = BenchArt(runs)

relabels = {
    'machine_instance_hostinfo_Hostname': 'host',
    'benchmark_config_scale': 'scale',
    'machine_instance_hostinfo_KernelRelease': 'kernel',
    'machine_disk_block_device_settings_nr_hw_queues': '#hwq',
    'machine_disk_block_device_settings_nr_requests': 'nr_requests',
    'machine_disk_block_device_settings_queue_depth': 'queue_depth',
    'machine_disk_block_device_settings_scheduler': 'scheduler',
    'postgres.gucs.set_gucs.huge_pages': 'huge_pages',
    'machine_disk_limits_size': 'disk_kind',
    'machine_disk_size_gb': 'disk_size_gb',
    'machine_instance_hostinfo_Hostname': 'host',
    'machine_instance_type': 'instance_type',
    'benchmark_config_scale': 'pgbench_scale',
}

ignores = ['machine_instance_hostinfo_KernelRelease',
                'machine_disk_block_device_settings_nr_hw_queues',
                'machine_disk_block_device_settings_nr_requests',
                'machine_disk_block_device_settings_queue_depth',
                'machine_disk_block_device_settings_scheduler']

occludes=set(['benchmark_config_time', 
              'postgres_load_data',
              'postgres_init',
              'postgres_version',
              'postgres_gucs_set_gucs_huge_pages',
              'machine_disk_limits_burst_iops',
              'machine_disk_trim',
              'machine_disk_limits_iops',
              'machine_disk_limits_burst_bw_mbps',
              'machine_disk_limits_bw_mbps',
              'machine_instance_limits_uncached_burst_iops',
              'machine_instance_limits_uncached_iops',
              'machine_instance_limits_uncached_burst_bw_mbps',
              'machine_instance_limits_uncached_bw_mbps',
              'machine_instance_mem_total_bytes',
             ])

figure = plt.figure(figsize=(15,20), constrained_layout=True)
figure.suptitle('pgbench')
figure.supylabel('tps', fontsize=16)

disk_identifier = ['machine_disk_size_gb', 'machine_disk_limits_size']


benchart.part(SubfigureRenderer(relabels), 'machine_instance_hostinfo_Hostname',
              'machine_instance_type', *disk_identifier)

benchart.part(SubfigureRenderer(relabels), 'machine_disk_caching')
benchart.part(SubfigureRenderer(relabels), 'benchmark_large_read')
benchart.part(SubfigureRenderer(relabels), 'benchmark_config_scale')

benchart.ignore(*ignores, *occludes)

timebound = 20
root, title = render(benchart, figure, timebound, relabels, occludes)
RenderJSON(title)